In [ ]:
%load_ext autoreload
%autoreload 2
import torch
from tqdm import tqdm
import numpy as np
import gymnasium as gym
import matplotlib.pyplot as plt
from IPython.display import Video

import sapien
import mani_skill.envs
from mani_skill.utils.wrappers import RecordEpisode

from vega_robot import Vega
from pick_single_ycb import PickSingleYCBEnv

np.set_printoptions(precision=2, suppress=True)
torch.set_printoptions(precision=2, sci_mode=False)

# Build Robot

In [ ]:
env = gym.make("Empty-v1", robot_uids="vega", render_mode="rgb_array")
env.action_space # shape (N, D)
env.observation_space # shape (N, ...)
env.reset()

# kf = env.agent.keyframes['rest']
# env.agent.robot.set_qpos(kf.qpos)
# env.agent.controller.reset()

plt.figure(figsize=(8, 8))
plt.imshow(env.render()[0].cpu().numpy())

In [ ]:
# 检查关节顺序
from mani_skill.envs.scene import ManiSkillScene  
from mani_skill.utils.building import URDFLoader  
  
loader = URDFLoader()  
loader.set_scene(ManiSkillScene())  
robot = loader.load("/home/xzhan742/Coding/ManiSkill/vega_grasp/dexmate-urdf/robots/humanoid/vega_1/vega_upper_body_mod.urdf")  
# robot = loader.load("/home/xzhan742/Coding/ManiSkill/mani_skill/assets/robots/panda/panda_v2.urdf")  
import rich
rich.print(list(enumerate(robot.active_joints_map.keys())))
# print("\n".join(list(robot.active_joints_map.keys())))

In [ ]:
env = gym.make("Empty-v1", robot_uids="vega",
                control_mode="pd_joint_delta_pos",
                render_mode="rgb_array")
# print(env.agent.robot.dof)
# print(env.agent.robot.active_joints)
# env.agent.robot.active_joints_map.keys()
# print(env.action_space)
# env.agent.controller
env.agent.robot.get_qvel().shape

In [ ]:
env.reset()
obs, rew, terminated, truncated, info = env.step(env.action_space.sample())
# obs (N, ...), rew (N, ), terminated (N, ), truncated (N, )
print(obs)
print(rew)
print(terminated)
print(truncated)
print(info)

In [ ]:
env = gym.make("Empty-v1", robot_uids="vega",
                control_mode="pd_joint_delta_pos",
                render_mode="rgb_array")
env = RecordEpisode(
    env,
    "./videos",
    max_steps_per_video=100
)

obs, _ = env.reset()
kf = env.agent.keyframes['rest']
env.agent.robot.set_qpos(kf.qpos)
env.agent.controller.reset()


for i in tqdm(range(200)):
    # action = np.ones(env.action_space.shape) * -0.1
    action = env.action_space.sample()
    action[:7] = 0
    # print(action)
    obs, reward, terminated, truncated, info = env.step(action)
    # print(obs)
env.close()

Video("./videos/0.mp4", embed=True, width=640) # Watch our replay

## Sensor

In [ ]:
env = gym.make("Empty-v1", robot_uids="vega", control_mode="pd_joint_delta_pos", obs_mode="rgb+depth+segmentation",
                render_mode="sensors")
# env.agent.robot.links_map['head_l3']
# env.agent.sensors.keys()
# env.print_sim_details()

env.agent._sensor_configs[3]

# obs, _ = env.reset()
# kf = env.agent.keyframes['rest']
# env.agent.robot.set_qpos(kf.qpos)

# plt.figure(figsize=(18, 8))
# plt.imshow(env.render()[0].cpu().numpy())

In [ ]:
# obs["sensor_data"].keys()
# obs["sensor_data"]["zed_left_camera"].keys()
env.agent.robot.links_map.keys()

In [ ]:
obs, _ = env.reset()
# kf = env.agent.keyframes['rest']
# env.agent.robot.set_qpos(kf.qpos)
obs, reward, terminated, truncated, info = env.step(np.zeros(env.action_space.shape))

n_cams = len(obs['sensor_data'])
plt.figure(figsize=(15, 5*n_cams))
env_id = 0
for i, (key, cam) in enumerate(obs['sensor_data'].items()):
    rgb, depth = cam['rgb'], cam['depth']
    plt.subplot(n_cams,3,1+3*i)
    plt.title(f"{key} - RGB")
    plt.imshow(rgb[env_id].cpu().numpy())
    plt.subplot(n_cams,3,2+3*i)
    plt.title(f"{key} - Depth")
    plt.imshow(depth[..., 0][env_id].cpu().numpy(), cmap="gray")
    plt.subplot(n_cams,3,3+3*i)
    plt.title(f"{key} - Segmentation")
    plt.imshow(cam["segmentation"][..., 0][env_id].cpu().numpy())
plt.tight_layout()

In [ ]:
env = gym.make("Empty-v1", robot_uids="vega", control_mode="pd_joint_delta_pos",
                render_mode="sensors")
env = RecordEpisode(
    env,
    "./videos",
    max_steps_per_video=100
)

obs, _ = env.reset()
kf = env.agent.keyframes['rest']
_k_qpos = kf.qpos.copy()
env.agent.robot.set_qpos(_k_qpos)
env.agent.controller.reset()


for i in tqdm(range(100)):
    _k_qpos[0] -= 0.01
    env.agent.robot.set_qpos(_k_qpos)
    # action = np.ones(env.action_space.shape) * -0.1
    action = env.action_space.sample()
    action[:7] = 0
    # print(action)
    obs, reward, terminated, truncated, info = env.step(action)
    # print(obs)
env.close()

Video("./videos/0.mp4", embed=True, width=1000) # Watch our replay

# Build Task

In [ ]:
env = gym.make("MyPickSingleYCB-v1", robot_uids="vega", control_mode="pd_joint_delta_pos", obs_mode="rgb+depth",
                render_mode="sensors")

obs, _ = env.reset()
kf = env.agent.keyframes['rest']
env.agent.robot.set_qpos(kf.qpos)

plt.figure(figsize=(18, 8))
plt.imshow(env.render()[0].cpu().numpy())

In [ ]:
%load_ext autoreload
%autoreload 2
import torch
from tqdm import tqdm
import numpy as np
import gymnasium as gym
import matplotlib.pyplot as plt
from IPython.display import Video

import sapien
import mani_skill.envs
from mani_skill.utils.wrappers import RecordEpisode

from vega_robot import Vega
from pick_single_ycb import PickSingleYCBEnv

np.set_printoptions(precision=2, suppress=True)
torch.set_printoptions(precision=2, sci_mode=False)

env = gym.make("MyPickSingleYCB-v1", robot_uids="vega", control_mode="pd_joint_delta_pos", obs_mode="rgb+depth",
                render_mode="rgb_array")

obs, _ = env.reset()
kf = env.agent.keyframes['rest']
env.agent.robot.set_qpos(kf.qpos)

plt.figure(figsize=(18, 8))
plt.imshow(env.render()[0].cpu().numpy())